In [ ]:
#calculate how many K clusters there should be 
wcss = [] 
for number_of_clusters in range(1, 30): 
    kmeans = KMeans(n_clusters = number_of_clusters, random_state = 42)
    kmeans.fit(scaled_x) 
    wcss.append(kmeans.inertia_)
wcss

ks = range(1, 30)
plt.plot(ks, wcss)
plt.axvline(18, linestyle='--', color='r')

In [ ]:
# initialize KMeans
kmeans = KMeans(n_clusters=18, random_state=0)
clusters = kmeans.fit(scaled_x)
labels = clusters.labels_
data_df['cluster'] = pd.Categorical(labels)


In [ ]:
score = silhouette_score(scaled_x, clusters.labels_, metric='euclidean')
print('Silhouetter Score: %.3f' % score)


In [ ]:
from sklearn import datasets
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(kmeans, k=(2,24))
 
visualizer.fit(scaled_x)        # Fit the data to the visualizer
visualizer.show() 


In [ ]:
range_n_clusters = [16, 18, 20, 22, 24, 26, 28, 30]
silhouette_avg = []
for num_clusters in range_n_clusters:  
  # initialise kmeans
  kmeans = KMeans(n_clusters=num_clusters)
  kmeans.fit(scaled_x)
  cluster_labels = kmeans.labels_
 
  # silhouette score
  silhouette_avg.append(silhouette_score(scaled_x, cluster_labels))
plt.plot(range_n_clusters,silhouette_avg)
plt.xlabel("Values of K") 
plt.ylabel("Silhouette score") 
plt.title("Silhouette analysis For Optimal k")
plt.show()

In [ ]:
visualizer = KElbowVisualizer(kmeans, k=(17, 24))
 
visualizer.fit(scaled_x)        # Fit the data to the visualizer
visualizer.show() 

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

fig, ax = plt.subplots(2, 2, figsize=(15,8))
for i in [2, 3, 4, 5]:
    '''
    Create KMeans instance for different number of clusters
    '''
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(i, 2)
    '''
    Create SilhouetteVisualizer instance with KMeans instance
    Fit the visualizer
    '''
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(scaled_x)